<a href="https://colab.research.google.com/github/subikkshas/DA6401/blob/main/DLass1q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 2 (10 Marks)
Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible such that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

In [7]:
import numpy as np
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

def one_hot(y, num_classes=10):
    return np.eye(num_classes)[y]

y_train, y_test = one_hot(y_train), one_hot(y_test)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))

class NeuralNetwork:
    def __init__(self, input_size=784, hidden_layers=[128, 64], output_size=10, learning_rate=0.01):
        self.layers = []
        self.biases = []
        self.learning_rate = learning_rate

        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.layers.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i+1])))

    def forward(self, x):
        self.a = [x]
        self.z = []

        for i in range(len(self.layers) - 1):
            z = np.dot(self.a[-1], self.layers[i]) + self.biases[i]
            self.z.append(z)
            self.a.append(relu(z))

        final_z = np.dot(self.a[-1], self.layers[-1]) + self.biases[-1]
        self.z.append(final_z)
        self.a.append(softmax(final_z))

        return self.a[-1]

    def backward(self, y_true):
        m = y_true.shape[0]
        dz = self.a[-1] - y_true

        for i in reversed(range(len(self.layers))):
            dw = np.dot(self.a[i].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m

            self.layers[i] -= self.learning_rate * dw
            self.biases[i] -= self.learning_rate * db

            if i > 0:
                dz = np.dot(dz, self.layers[i].T) * relu_derivative(self.z[i-1])

    def train(self, x_train, y_train, epochs=10, batch_size=32):
        for epoch in range(epochs):
            x_sample, y_sample = x_train[:1], y_train[:1]
            y_pred = self.forward(x_sample)
            print("Y Hat (Single Example):", y_pred)
            self.backward(y_sample)

            y_pred_train = self.forward(x_train)
            loss = cross_entropy_loss(y_pred_train, y_train)
            accuracy = np.mean(np.argmax(y_pred_train, axis=1) == np.argmax(y_train, axis=1))

            print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

nn = NeuralNetwork(hidden_layers=[128, 64], learning_rate=0.01)
nn.train(x_train, y_train, epochs=10, batch_size=32)


Y Hat (Single Example): [[0.10000889 0.10005186 0.10008474 0.09999062 0.09990906 0.09996893
  0.10005077 0.1000124  0.09994895 0.09997379]]
Epoch 1: Loss = 2.3026, Accuracy = 0.1000
Y Hat (Single Example): [[0.09990659 0.09994996 0.09998358 0.09988935 0.09980724 0.09986501
  0.09994992 0.0999122  0.09984952 0.10088664]]
Epoch 2: Loss = 2.3027, Accuracy = 0.1000
Y Hat (Single Example): [[0.09980362 0.09984732 0.09988182 0.09978729 0.0997046  0.09976046
  0.09984857 0.09981126 0.0997493  0.10180576]]
Epoch 3: Loss = 2.3027, Accuracy = 0.1000
Y Hat (Single Example): [[0.09969994 0.09974399 0.09977937 0.0996845  0.09960121 0.09965518
  0.09974651 0.09970961 0.09964833 0.10273134]]
Epoch 4: Loss = 2.3027, Accuracy = 0.1000
Y Hat (Single Example): [[0.09959575 0.09964069 0.09967668 0.0995808  0.09949703 0.09954947
  0.09964367 0.09960709 0.0995463  0.10366252]]
Epoch 5: Loss = 2.3027, Accuracy = 0.1000
Y Hat (Single Example): [[0.09949089 0.09953672 0.09957321 0.09947674 0.09939211 0.0994430